In [2]:
import pandas as pd
import os
import re
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import sys
from datetime import datetime


In [3]:
def log_message(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    sys.stdout.write(f"[{timestamp}] {message}\n")
    sys.stdout.flush()

In [4]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "data")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:

# 모델과 토크나이저 로드
model_name = 'yanolja/EEVE-Korean-Instruct-10.8B-v1.0'
log_message(f"Loading tokenizer and model: {model_name}")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(DEVICE)
log_message("Model loaded successfully")


[2024-11-05 15:03:07] Loading tokenizer and model: yanolja/EEVE-Korean-Instruct-10.8B-v1.0


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

[2024-11-05 15:03:20] Model loaded successfully


# 뉴스기사 제목 생성

In [ ]:

data_path = os.path.join(DATA_DIR, "train.csv")
log_message(f"Loading data from: {data_path}")
origin_df = pd.read_csv(data_path)
new_df = origin_df.copy()
for idx, text in enumerate(new_df['text']) : 
    new_df.loc[new_df['text'] == text, 'text'] = re.sub(r'[^가-힣ㅏ-ㅣ ]+', ' ', text).strip()

log_message(f"Data loaded. Shape: {new_df.shape}")
log_message(f"Model: {model_name}")
log_message(f"Data length: {len(new_df)}")

for tidx, noised_text in tqdm(enumerate(new_df['text']), total=len(new_df), desc="Processing"):
    if tidx % 10 == 0:  # Log every 10 iterations
        log_message(f"Processing item {tidx}/{len(new_df)}")
    
    prompt = f"""당신은 한국어 신문 기사 헤드라인 생성 전문가입니다. 주어진 불완전한 문자들을 포함하는 새로운 신문 기사 헤드라인을 생성하세요. 
                주어진 불완전한 문장의 문자들을 가장 많이 포함하는 자연스러운 신문기사 헤드라인을 생성하세요. 생성 시 다음 지침을 따르세요:

                1. 한국어 신문 기사 헤드라인의 일반적인 스타일과 어조를 유지하세요.
                2. 기존의 문장의 단어와 문자를 최대한 많이 포함해야합니다.
                3. 문맥을 고려하여 가장 그럴듯한 내용으로 복구하세요.
                4. 생성된 헤드라인은 완전하고 의미가 통해야 합니다.
                5. 주어진 문장이 많이 훼손되어 복구가 어려운 경우 완전히 새로운 헤드라인을 생성하세요.
                
                이제 다음의 문장을 이용해 새로운 헤드라인을 생성해주세요:
                불완전한 헤드라인: {noised_text}
                생성된 헤드라인:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_text = generated_text.split("생성된 헤드라인:")[-1].split('\n')[0]
    new_df.loc[tidx, 'text'] = cleaned_text

output_path = './data/fill_mask/train_generated.csv'
log_message(f"Saving results to: {output_path}")
new_df.to_csv(output_path, index=False)
log_message("Process completed successfully")

[2024-11-05 02:30:25] Loading data from: /data/ephemeral/home/kjh/data/train.csv
[2024-11-05 02:30:27] Data loaded. Shape: (2800, 3)
[2024-11-05 02:30:27] Model: yanolja/EEVE-Korean-Instruct-10.8B-v1.0
[2024-11-05 02:30:27] Data length: 2800


Processing:   0%|          | 0/2800 [00:00<?, ?it/s]

[2024-11-05 02:30:27] Processing item 0/2800


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing:   0%|          | 0/2800 [00:00<?, ?it/s]


KeyboardInterrupt: 

# Augmentation

In [13]:
data_path = os.path.join(DATA_DIR, "fill_mask/train_generated.csv")
log_message(f"Loading data from: {data_path}")
origin_df = pd.read_csv(data_path)
new_df = origin_df.copy()
new_df.head()

[2024-11-05 08:21:48] Loading data from: /data/ephemeral/home/kjh/data/fill_mask/train_generated.csv


,ID,text,target
0,ynat-v1_train_00000,정권이 파리 미사 참석에 이용기간 제한을 두고 분열을 보임,4
1,ynat-v1_train_00001,찰 국 로한 회장이 맡은 주요 프로젝트,3
2,ynat-v1_train_00002,"""김정은, 자주통일 새 열쇠를 잡고 나서야 한다""",2
3,ynat-v1_train_00003,"""갤럭시 노트 7, 주말 동안 10만 대 개통…시장 불법 보조금 논란에 얼룩져""",5
4,ynat-v1_train_00004,"""대선을 앞두고 감시 강화, 단발적 비해""",6


In [ ]:
log_message(f"Data loaded. Shape: {new_df.shape}")
log_message(f"Model: {model_name}")
log_message(f"Data length: {len(new_df)}")

batch_size = 8  # 적절한 배치 크기를 설정하세요. GPU 메모리에 따라 조정이 필요할 수 있습니다.

for i in tqdm(range(0, len(new_df), batch_size), desc="Processing batches"):
    batch = new_df['text'][i:i+batch_size].tolist()
    
    prompts = [f"""당신은 한국어 신문 기사 헤드라인 수정 및 생성 전문가입니다. 주어진 문장을 이용해 헤드라인을 생성하세요. 
                주어진 문장과 유사한 의미를 가지는 자연스러운 신문기사 헤드라인을 생성하세요. 생성 시 다음 지침을 따르세요:

                1. 한국어 신문 기사 헤드라인의 일반적인 스타일과 어조를 유지하세요.
                2. 기존의 문장의 주제와 유사한 주제의 헤드라인을 생성해주세요.
                3. 생성된 헤드라인은 완전하고 의미가 통해야 합니다.
                4. 주어진 문장이 많이 훼손되어 복구가 어려운 경우 완전히 새로운 헤드라인을 생성하세요.
                
                이제 다음의 문장을 이용해 새로운 헤드라인을 생성해주세요:
                불완전한 헤드라인: {title_text}
                생성된 헤드라인:""" for title_text in batch]

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,  # 헤드라인은 짧으므로 max_new_tokens를 줄였습니다.
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    cleaned_texts = [text.split('생성된 헤드라인:')[-1].split("\n")[0].strip() for text in generated_texts]
    for text in cleaned_texts :
        print(text)
    new_df.loc[i:i+batch_size-1, 'text'] = cleaned_texts

output_path = './data/generate_context/context_generated.csv'
log_message(f"Saving results to: {output_path}")
new_df.to_csv(output_path, index=False)
log_message("Process completed successfully")

# Cleaning

In [7]:
data_path = os.path.join(DATA_DIR, "Self_train/self_train_step2.csv")
log_message(f"Loading data from: {data_path}")
origin_df = pd.read_csv(data_path)
new_df = origin_df.copy()
new_df.head()
len(new_df)

[2024-11-05 15:03:48] Loading data from: /data/ephemeral/home/kjh/data/Self_train/self_train_step2.csv


2800

In [ ]:
log_message(f"Data loaded. Shape: {new_df.shape}")
log_message(f"Model: {model_name}")
log_message(f"Data length: {len(new_df)}")

batch_size = 8  # 적절한 배치 크기를 설정하세요. GPU 메모리에 따라 조정이 필요할 수 있습니다.

for i in tqdm(range(0, len(new_df), batch_size), desc="Processing batches"):
    batch = new_df['text'][i:i+batch_size].tolist()
    
    prompts = [f"""당신은 한국어 뉴스 헤드라인 정제 전문가입니다. 주어진 텍스트에서 노이즈를 제거하고 자연스러운 뉴스 헤드라인을 생성해야 합니다. 다음 지침을 따르세요:

                    1. 불필요한 특수문자, 숫자, 알파벳을 제거하세요.
                    2. 맞춤법과 문법을 교정하세요.
                    3. 의미가 불분명한 부분은 문맥을 고려하여 자연스럽게 보완하세요.
                    4. 원래 헤드라인의 주요 키워드와 의미를 최대한 유지하세요.
                    5. 출력은 정제된 헤드라인만 포함해야 합니다.

                    예시:
                    입력: "157 조우 선동, 독과 함께 PAG 5전이 고군분투",1
                    출력: 조우선 선수, 독일 팀과 함께 PGA 투어 5차전에서 고군분투

                    이제 다음 헤드라인을 정제해주세요:
                    입력: {title_text}
                    출력: """ for title_text in batch]

    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,  # 헤드라인은 짧으므로 max_new_tokens를 줄였습니다.
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    cleaned_texts = [text.split('이제 다음 헤드라인을 정제해주세요:')[-1].split("출력:")[-1].split('\n')[0].strip() for text in generated_texts]
    for text in cleaned_texts :
        print(text)
    new_df.loc[i:i+batch_size-1, 'text'] = cleaned_texts

output_path = './data/generate_context/context_generated.csv'
log_message(f"Saving results to: {output_path}")
new_df.to_csv(output_path, index=False)
log_message("Process completed successfully")



[2024-11-05 15:06:39] Data loaded. Shape: (2800, 3)
[2024-11-05 15:06:39] Model: yanolja/EEVE-Korean-Instruct-10.8B-v1.0
[2024-11-05 15:06:39] Data length: 2800


Processing batches:   0%|          | 0/350 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batches:   0%|          | 1/350 [00:18<1:48:01, 18.57s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


2분 KT 이용 기간이 종료됩니다.
文 대통령이 로 LH공사 사장에게 2시간 면담을 요청하다
김정은, 새해 자주통일 메시지 발표
갤럭시 노트 8, 주말에 27만대 개통...시장은 불법 보조금에 얼룩져
2020년 미국 대선을 앞두고 감시 강화, R2F 단발에 비해
6명 중 1명의 미국 성인, 배우자/연인의 빚 떠안다
롯데와 기아, 광주 경기 취소
33득점 아가메즈, 우리카드, KB손해보험 꺾고 3위 확보


Processing batches:   1%|          | 2/350 [00:52<2:40:37, 27.69s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


朴 대통령, 경주 지진 현장 방문... 충격받은 모습
듀얼심 아이폰, 하반기 출시 기대감 커져…알뜰폰 이용자를 위한
르모델 열화상 카메라 주가 상승
1월 옵션 만기일 NH투자 매도 우세
황 총리, 모든 부처에 비상 대비 태세 철저히 준비 주문
엑스아이 수리: 디디에프레 41-0
2대 문인당, 동2구 4동 민관2동 7사위 철거
KISA 박민정 책임연구원, APTLD 이사로 선출


...

Processing batches:   3%|▎         | 10/350 [04:53<2:38:01, 27.89s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


독일, 반이스라엘 테러 팔레스타인인 강연 금지 및 비자 취소

9U 천재 사령탑 O오 감독, 한국전에서 승리
6호 특공대, G&C 캠페인을 위한 52억 기부금 신고
北, "조국통일" 민주전선 결성
롯데백화점에 아마존, 3천만 달러 투자
2월과 3월 대구·경북 고용유지지원금 신청 급증, 7,550건
4,500대 생산, 본트카 애스턴마틴 스파 국내


Processing batches:   3%|▎         | 10/350 [05:03<2:51:52, 30.33s/it]


KeyboardInterrupt: 

: 